In [2]:
from main import *
import time
import pandas as pd
import json
import requests
import os
from util import *
api_key = os.environ.get("riot_api_key")
program = GameDataGenerator(api_key, name=None)
_util = util()


Goal: 
to read 10k rows from csv X<br>
send them through main 100 entries at a time - NEED TO FIGURE OUT HOW TO KEEP TRACK OF WHERE I AM IN THE TOTAL LIST (INDEXING PROBLEM)<br>
piece together all 100 runs into one big dataset<br>
output dataset to be split into training and testing data<br>

1. get info: check if ranked then take data
2. save to local cache file
3. check limit, if okay then back to step 1

In [38]:
""" Trying new source of match ids (~140000 NA1 matches) """
file = open("americas_match_ids.json", "r")
data = json.load(file)


It exists already


In [40]:
"""Loading data to cache from json source"""
EXPECTED_RUNTIME = 1.2
for id in data['match_ids']:
    # set start time to compare later
    startTime = time.time()
    
    # check if id is already in cache
    cachedResponse = _util.loadFromCache(cacheKey=id)

    # if it's found, skip this id
    if cachedResponse is not None: 
        continue

    # id wasn't found, so make call
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{id}?api_key={api_key}"
    r = requests.get(url)
    if r.status_code != 200:
        continue
    gameData = program.getOneGameStatsWithResponse(r)
    
    # save call data to cache
    _util.saveToCache(gameData, id)

    # check if it ran faster than 1.2 sec 
    # because if it did, need to stall a bit 
    # to not exceed more than 1 call per 1.2 sec
    elaspedTime = time.time() - startTime
    if elaspedTime < EXPECTED_RUNTIME:
        time.sleep(EXPECTED_RUNTIME - elaspedTime)

    # if it gets close to the 100 call limit, it will automatically rest for 2 minutes
    if program.exceedsRateLimit(dict(r.headers)):
        progress = len(os.listdir(_util.cacheDir))
        print(f"Taking a 2 minute break but currently {progress}/143177")
        time.sleep(120)
        print("Resuming...")



<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\diepw\AppData\Local\Temp\ipykernel_31424\4152742480.py:17: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if r.status_code is not 200:


Taking a 2 minute break but currently 1482/143177
Taking a 2 minute break but currently 1578/143177
Taking a 2 minute break but currently 1674/143177


In [3]:
len(os.listdir(_util.cacheDir))

NameError: name '_util' is not defined

for each dict in cache
if dict name not in csv first col
then add dict to csv

In [ ]:
# open current contents of the csv
try:
    dataFromCSV = pd.read_csv("data.csv")
except:
    print("The csv was empty so the headers were intialized.")
    temp = _util.loadFromCache("NA1_4794682594")
    columnNames = temp.keys()
    temp_df = pd.DataFrame(columns=columnNames)
    temp_df.to_csv("data.csv", index=False)


# create a list of current ids stored to check for incoming dupes
gameIds = dataFromCSV['gameId'].tolist()
# loop through each thing cached, filter out dupes, and then filter out non ranked. lastly, append to csv
for item in os.listdir(_util.cacheDir):
    item = item.split('.')[0]
    if item not in gameIds:
        itemInfo = _util.loadFromCache(item)
        if itemInfo['queueId'] == 420:
            df = pd.DataFrame([itemInfo])
            df.to_csv("data.csv", mode="a", header=False, index=False)
        # else:
        #     print("it wasnt a ranked game")